<a href="https://colab.research.google.com/github/EddyBautista-93/Intro-To-AI-Agents/blob/main/Intro_to_ai_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to AI Agents - EventForge

For this workshop we will be creating a AI Agent with the ability to help us plan for a event we will be hosting. \
Below is the following tools we will be looking building together.

Tool # | Name | What it adds
--- | --- | ---
1 | DuckDuckGoSearch | Venue Discovery - Built In
2 | Image Generation | Create Promo Visual - Community Tool
3 | Agenda Generator | Custom Schedule Based On evenr type - Custom
4 | Budget Estimator | Financial planning for headcount, food, venue, and swag - Custom
5 | Post Gen | Generate Post

# Getting started

We’ll begin by installing the dependencies needed for building our smart event planning agent.

In [ ]:
! pip install smolagents gradio

The Installs


*   **smolagents**: A lightweight framework for building LLM-powered agents with tool calling support.
*   **gradio**: A UI library for interactive web demos in notebooks



To use models like Mixtral from Hugging Face, you’ll need to authenticate:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


To create a smart AI agent with SmolAgents, you need two core things:


*   A model – the brain of the agent that generates and understands text
*   A set of tools – external functions that the model can call to extend its capabilities

## The Model
Text generation models power your agent. Think of the model as the engine that drives everything forward. SmolAgents supports multiple model types depending on how you want to run your agent.

### Model Options
**TransformerModel** \
Runs a locally initialized transformers pipeline. Ideal for models you have downloaded or finetuned.

**HFApiModel**(What we'll use) \
Uses Hugging Face Inference API via huggingface_hub.InferenceClient to call hosted models. Think LLaMA and Mixtral

**LiteLLM**
Suppoers calling 100+ LLMs through the LiteLLM router-userful for OpenAI, claude, etc with unified configs.

**AzureOpenAiServerModel** \
Connects to OpenAI models deployed through Microsoft Azure

**MLXModel** \
Creates a mlx-lm pipeline that run inference on your local machine.


# Tools
Tools give your AI agent ***agency*** - they let thge model take actions beyond just talking.\
Tools are passed in like this:
``` python
tools=[]
```
These tools can incluse things like:


*   Search Engines
*   Image Generators
*   Custom Tools
*   RAG (Retrieval Augmented Generation ) Functions

## Creating the Agent
Once you have the model and tools, you can create a agent like this:

``` python
model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = HfApiModel(model_id=model_id) # By default if you don't pass in a model id the hfapi it uses the qwen model
agent = ToolCallingAgent(tools=[], model=model)
```

# ToolCallingAgent

The ToolCallingAgent is a lightweight class designed to let large language models interact with tools.

Since we're building an agent that plans events, we'll focus on this agent type to give the model the power to search the web, generate images, generate post and estimate budgets.

Key Features of ToolCallingAgent
- **Contextual Memory**: Maintains a chat history, allowing for ongoing reasoning and continuity across multiple prompts..
- **Multi-Step Reasoning**: ToolCallingAgents follow a "think-act-observe" cycle as part of the ReAct framework.
- **Customizability**: You can define:
  * A list of tools for the agent to use.
  * Specify prompt templates.
  * Adjust other parameters like planning intervals during initialization



In short: ToolCallingAgent is what gives your AI brain the power to do more than talk—it can take action.

## Tool 1: DuckDuckGoSearch + Location
The first tool we're using is built directly into the SmolAgents framework: DuckDuckGoSearchTool.

This tools allows the agent to search the web in real time and retrieve up-to-date information. Perfect for finding venues, vendors, and other real-world event planning needs.

This tool:
* Searches DuckDuckGo for relevant results.
* Works out of the box - no API key needed.
* Can be combined with custom tools (like a location preset)

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel , DuckDuckGoSearchTool # we introduce a built in tool

model = HfApiModel() # You can choose to not pass any model_id to HfApiModel to use a default free model
# you can also specify a particular provider e.g. provider="together" or provider="sambanova"

# @tool
# def location() -> str:
#   """
#   Returns the location of the user
#   """
#     return "San Antonio, Texas"

web_search = DuckDuckGoSearchTool()

agent = ToolCallingAgent(tools=[web_search], model=model)
agent.run("Whats a good indoor venue for a tech focused event with free wifi in San Antonio ")

## Tool 2: Image Generation

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel , DuckDuckGoSearchTool , load_tool
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

agent = ToolCallingAgent(tools=[image_generation_tool], model=model)
agent.run("Provide minimal line art promo art for my event, the theme is automating boring things with AI Agents.")

## Tool 4: Post Generation

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, tool

model = HfApiModel(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

@tool
def generate_post(event_name: str, date: str, location: str, highlight: str = "networking and learning") -> str:
    """
    Generates a LinkedIn post for a tech event.

    Args:
        event_name: The name of the event.
        date: The date of the event.
        location: The city or venue where the event is held.
        highlight: What's special about the event. Defaults to 'networking and learning'.
    """
    agent = ToolCallingAgent(model=model, tools=[])
    prompt = (
        f"Write a LinkedIn post promoting an event.\n\n"
        f"Event Name: {event_name}\n"
        f"Date: {date}\n"
        f"Location: {location}\n"
        f"Highlight: {highlight}\n\n"
        "Make it sound professional, friendly, and engaging. Add relevant hashtags."
    )
    return agent.run(prompt)

In [ ]:
agent = ToolCallingAgent(tools=[generate_post], model=model)

response = agent.run("Create a LinkedIn post for an event called FutureTech SA happening April 25 at Geekdom focused on AI agents.")
print(response)

## Tool 5: Estimate Budget

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, tool


@tool
def estimate_budget(guests: int, catering: bool = True, venue: bool = True, swag: bool = False) -> dict:
    """
    This tool estimates the total cost of an event.

    Args:
        guests: The number of guests attending the event.
        catering: Whether catering is included. Defaults to True.
        venue: Whether a venue is included. Defaults to True.
        swag: Whether swag is included. Defaults to False.
    """
    base_cost = 0
    if catering:
        base_cost += guests * 15  # $15 per guest for food
    if venue:
        base_cost += 500  # Flat rate for venue
    if swag:
        base_cost += guests * 5  # $5 per guest for merch
    return {
        "Estimated Total": f"${base_cost}",
        "Breakdown": {
            "Catering": f"${guests * 15 if catering else 0}",
            "Venue": "$500" if venue else "$0",
            "Swag": f"${guests * 5 if swag else 0}"
        }
    }

agent = ToolCallingAgent(tools=[estimate_budget], model=model)
agent.run("How much will 20 guest cost me for this event")

# Exercise: Custom Tools

In [ ]:
from smolagents import ToolCallingAgent, HfApiModel, tool


@tool
def custom_tool_1() -> str:
    """
    This tool provides the guest list for an event.
    """
    return "This tool does nothing."


@tool
def custom_tool_2() -> str:
    """
    This tool provides the guest list for an event.
    """
    return "This tool does nothing."




Bringing it all together

In [ ]:

import gradio as gr
from smolagents import ToolCallingAgent, HfApiModel


tool_sets = {
    "Custom Tools": [custom_tool_1, custom_tool_2],
    "Search Only": [web_search],
    "Post + Image": [generate_post, image_generation_tool],
    # "Full Power": [web_search, image_generation_tool, generate_agenda, estimate_budget]
}

# 4. Run agent
def run_agent(prompt, tool_choice):
    tools = tool_sets.get(tool_choice, [])
    agent = ToolCallingAgent(model=model, tools=tools)
    response = agent.run(prompt)
    return response

# 5. Gradio UI
gr.Interface(
    fn=run_agent,
    inputs=[
        gr.Textbox(label="Your Event Planning Prompt", placeholder="e.g. Plan a tech meetup in San Antonio"),
        gr.Radio(choices=list(tool_sets.keys()), label="Choose Tool Set", value="No Tools")
    ],
    outputs=gr.Textbox(label="Agent Response"),
    title="🧠 Event Planner AI Agent",
    description="Explore how an AI agent becomes smarter and more helpful as you add new tools!"
).launch(share=True)